# Training SOTA CNNs

## Quickvision + PyTorch Image Models CNNs

- A Huge home to SOTA CNNs is [PyTorch Image Models](https://github.com/rwightman/pytorch-image-models) by Ross Wightman.

- It has high quality models with ImageNet weights.

- Let's train these for Trasnfer Learning Tasks !


In [ ]:
# Install PyTorch Image Models
! pip install -q timm
! pip install -q git+https://github.com/Quick-AI/quickvision.git

In [18]:
import torch
import torch.optim as optim
from torch.nn import functional as F
from tqdm import tqdm
from torch import nn
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as T

- A List of PreTrained Models Provided By Timm

In [ ]:
import timm
from pprint import pprint
model_names = timm.list_models(pretrained=True)
pprint(model_names)

- Let's stick to EfficientNet and Train it !

## CIFAR10 Dataset and Data Loader

- We use CIFAR10 Dataset to train on.
- It is directly available in torchvision

In [4]:
TRAIN_BATCH_SIZE = 512  # Training Batch Size
VALID_BATCH_SIZE = 512  # Validation Batch Size

In [5]:
train_transforms = T.Compose([T.ToTensor(), T.Normalize((0.5,), (0.5,))])
valid_transforms = T.Compose([T.ToTensor(), T.Normalize((0.5,), (0.5,))])

In [6]:
train_dataset = torchvision.datasets.CIFAR10("./data", download=True, train=True, transform=train_transforms)
valid_dataset = torchvision.datasets.CIFAR10("./data", download=True, train=False, transform=valid_transforms)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
train_loader = DataLoader(train_dataset, TRAIN_BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, VALID_BATCH_SIZE, shuffle=False)
VALID_BATCH_SIZE = 512  # Validation Batch Size

## Train EfficientNet from PyTorch Image Models

In [ ]:
model = timm.create_model("efficientnet_b0", pretrained=True, num_classes=10, in_chans=3)

In [9]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

### Here is where you can use Quickvision's Training Recipe to train these models

In [10]:
from quickvision.models.classification import cnn

In [11]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [12]:
history = cnn.fit(model=model, epochs=2, train_loader=train_loader,
                  val_loader=valid_loader, criterion=criterion, device=device,
                  optimizer=optimizer)

  0%|          | 0/2 [00:00<?, ?it/s]


Training Epoch = 0
Batch Train Time: 0.522 (0.522)  Loss:  7.6988 (7.6988)  Top 1 Accuracy: 10.5469 (10.5469)  Top 5 Accuracy: 51.5625 (51.5625)
Batch Train Time: 0.143 (0.200)  Loss:  1.1475 (2.0493)  Top 1 Accuracy: 61.6071 (45.3340)  Top 5 Accuracy: 95.5357 (87.4720)
Time taken for train step = 19.60645031929016 sec

Validating Epoch = 0
Batch Inference Time: 0.111 (0.111)  Loss:  1.1467 (1.1467)  Top 1 Accuracy: 58.2031 (58.2031)  Top 5 Accuracy: 96.2891 (96.2891)


 50%|█████     | 1/2 [00:32<00:32, 32.14s/it]

Batch Inference Time: 0.074 (0.115)  Loss:  1.3251 (1.1606)  Top 1 Accuracy: 51.1029 (60.1800)  Top 5 Accuracy: 95.5882 (95.9100)
Finished the validation epoch
Time taken for validation step = 2.2983126640319824 sec
Done Training, Model Saved to Disk

Training Epoch = 1
Batch Train Time: 0.222 (0.222)  Loss:  1.0024 (1.0024)  Top 1 Accuracy: 66.4062 (66.4062)  Top 5 Accuracy: 96.4844 (96.4844)
Batch Train Time: 0.162 (0.198)  Loss:  0.8628 (0.8909)  Top 1 Accuracy: 71.4286 (69.1740)  Top 5 Accuracy: 97.3214 (97.5800)
Time taken for train step = 19.38525390625 sec

Validating Epoch = 1
Batch Inference Time: 0.133 (0.133)  Loss:  0.9281 (0.9281)  Top 1 Accuracy: 70.1172 (70.1172)  Top 5 Accuracy: 97.8516 (97.8516)


100%|██████████| 2/2 [00:53<00:00, 26.97s/it]

Batch Inference Time: 0.069 (0.120)  Loss:  1.0122 (0.8934)  Top 1 Accuracy: 62.5000 (69.4400)  Top 5 Accuracy: 98.1618 (97.7600)
Finished the validation epoch
Time taken for validation step = 2.3908989429473877 sec
Done Training, Model Saved to Disk


- In return you get a Keras Like History Dictionary.
- It keeps a track of Metrics

In [14]:
pprint(history)

{'train': {'loss': [2.049303819618225, 0.8909260382080079],
           'top1_acc': [45.33400000366211, 69.17400001464844],
           'top5_acc': [87.47199999267578, 97.58000003662109]},
 'val': {'loss': [1.1606252620697022, 0.8934034944534301],
         'top1_acc': [60.17999995727539, 69.44],
         'top5_acc': [95.90999996337891, 97.7600000366211]}}


- You can also get a granular control over these using `train_step`, `val_step` methods.


In [19]:
EPOCHS = 2

for epoch in tqdm(range(EPOCHS)):
    print()
    print(f"Training Epoch = {epoch}")
    train_metrics = cnn.train_step(model, train_loader, criterion, device, optimizer)
    print()

    print(f"Validating Epoch = {epoch}")
    valid_metrics = cnn.val_step(model, valid_loader, criterion, device)

  0%|          | 0/2 [00:00<?, ?it/s]


Training Epoch = 0
Batch Train Time: 0.326 (0.326)  Loss:  0.6633 (0.6633)  Top 1 Accuracy: 78.1250 (78.1250)  Top 5 Accuracy: 99.0234 (99.0234)
Batch Train Time: 0.135 (0.200)  Loss:  0.7350 (0.6222)  Top 1 Accuracy: 74.4048 (78.3520)  Top 5 Accuracy: 99.4048 (98.9080)
Time taken for train step = 19.601767778396606 sec

Validating Epoch = 0
Batch Inference Time: 0.109 (0.109)  Loss:  0.7944 (0.7944)  Top 1 Accuracy: 72.2656 (72.2656)  Top 5 Accuracy: 98.4375 (98.4375)


 50%|█████     | 1/2 [00:21<00:21, 21.92s/it]

Batch Inference Time: 0.071 (0.114)  Loss:  0.8842 (0.8489)  Top 1 Accuracy: 68.0147 (71.6100)  Top 5 Accuracy: 98.5294 (97.9200)
Finished the validation epoch
Time taken for validation step = 2.2907419204711914 sec

Training Epoch = 1
Batch Train Time: 0.231 (0.231)  Loss:  0.4109 (0.4109)  Top 1 Accuracy: 86.5234 (86.5234)  Top 5 Accuracy: 99.8047 (99.8047)
Batch Train Time: 0.139 (0.201)  Loss:  0.5512 (0.4279)  Top 1 Accuracy: 82.1429 (85.1920)  Top 5 Accuracy: 99.7024 (99.5020)
Time taken for train step = 19.730385780334473 sec

Validating Epoch = 1
Batch Inference Time: 0.111 (0.111)  Loss:  0.8637 (0.8637)  Top 1 Accuracy: 72.4609 (72.4609)  Top 5 Accuracy: 98.8281 (98.8281)


100%|██████████| 2/2 [00:43<00:00, 21.98s/it]

Batch Inference Time: 0.068 (0.115)  Loss:  0.8986 (0.8965)  Top 1 Accuracy: 70.2206 (73.1100)  Top 5 Accuracy: 98.1618 (97.8400)
Finished the validation epoch
Time taken for validation step = 2.298086404800415 sec


- Again, quickvision computes metrics for you !
- You can print them to have look !

In [20]:
pprint(train_metrics)

OrderedDict([('loss', 0.42793877683639525),
             ('top1', 85.19200002197266),
             ('top5', 99.50200002685547)])


In [22]:
pprint(valid_metrics)

OrderedDict([('loss', 0.8965282435417176),
             ('top1', 73.11000001220702),
             ('top5', 97.84000003662109)])
